# Parameters

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../..')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import datetime

import numpy as np
import pandas as pd

from src.data import train_test_split, MRISequence
from src.model import create_model, compile_model, load_checkpoint
from src.model.evaluation import show_metrics

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['image.cmap'] = 'viridis'

%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 15})

In [3]:
import tensorflow as tf

# RANDOM_SEED = 250398
# tf.random.set_seed(RANDOM_SEED)

print(tf.version.VERSION)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.2
Num GPUs Available:  1


## Setup

In [4]:
%%time

ROOT_DIR = '../../../tmp'
DEFAULT_CHECKPOINT_DIRECTORY_LOCAL = os.path.join(ROOT_DIR, 'checkpoints')
DEFAULT_BCKP_CHECKPOINT_DIRECTORY_LOCAL = os.path.join(ROOT_DIR, 'bckp-checkpoints')

LOG_DIRECTORY = os.path.join(ROOT_DIR, 'logs')
CHECKPOINT_DIRECTORY = DEFAULT_CHECKPOINT_DIRECTORY_LOCAL

LOG_DIRECTORY_LOCAL = LOG_DIRECTORY
CHECKPOINT_DIRECTORY_LOCAL = CHECKPOINT_DIRECTORY

DATA_DIR_NAME = 'data-v3'
DATA_DIR = os.path.join(ROOT_DIR, DATA_DIR_NAME)

saliencies_and_segmentations_v2_path = os.path.join(ROOT_DIR, 'saliencies_and_segmentations_v2')

if not os.path.exists(CHECKPOINT_DIRECTORY):
    os.mkdir(CHECKPOINT_DIRECTORY)

if not os.path.exists(LOG_DIRECTORY):
    os.mkdir(LOG_DIRECTORY)

val = False

class_names = ['AD', 'CN']

# get paths to data
train_dir, test_dir, val_dir = train_test_split(
    saliencies_and_segmentations_v2_path,
    ROOT_DIR,
    split=(0.8, 0.15, 0.05),
    dirname=DATA_DIR_NAME)

# set the batch size for mri seq
batch_size = 12
input_shape = (104, 128, 104, 1) # (112, 112, 105, 1)
resize_img = True
crop_img = True

# if y is one-hot encoded or just scalar number
one_hot = True

# class weightss (see analysis notebook)
class_weights = {0: 0.8072289156626505, 1: 1.3137254901960784}

# description statistics of the dataset
desc = {'mean': -3.6344006e-09, 'std': 1.0000092, 'min': -1.4982183, 'max': 10.744175}

if 'desc' not in locals():
    print('initializing desc...')
    desc = get_description(MRISequence(
        train_dir,
        64,
        class_names=class_names,
        input_shape=input_shape),
        max_samples=None)
    print(desc)


normalization={ 'type':'normalization', 'desc': desc }
# normalization={'type':'standardization', 'desc':desc }

augmentations = None
augmentations_inplace = True
# enable augmentations in mri seq (otherwise it can be enabled in dataset)
# augmentations={ 'random_swap_hemispheres': 0.5 }

# initialize sequences
print('initializing train_seq...')
train_seq = MRISequence(
    train_dir,
    batch_size,
    class_names=class_names,
    augmentations=augmentations,
    augmentations_inplace=augmentations_inplace,
    input_shape=input_shape,
    resize_img=resize_img,
    crop_img=crop_img,
    one_hot=one_hot,
    class_weights=class_weights,
    normalization=normalization)

print('initializing test_seq...')
test_seq = MRISequence(
    test_dir,
    batch_size,
    class_names=class_names,
    input_shape=input_shape,
    resize_img=resize_img,
    crop_img=crop_img,
    one_hot=one_hot,
    normalization=normalization)

if val:
    print('initializing val_seq...')
    val_seq = MRISequence(
        val_dir,
        batch_size,
        class_names=class_names,
        input_shape=input_shape,
        resize_img=resize_img,
        crop_img=crop_img,
        one_hot=one_hot,
        class_weights=class_weights,
        normalization=normalization)
else:
    print('val_seq = test_seq')
    val_seq = test_seq

model_key = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = os.path.join(LOG_DIRECTORY, model_key)
print(f'log_dir: {log_dir}')

not copying files since the destination directory already exists
initializing train_seq...
initializing test_seq...
val_seq = test_seq
log_dir: ../../../tmp\logs\20210509-105142
Wall time: 0 ns


## Analysis

Each experiment consisted of 10 images, 5 TP and 5 TN.

In [5]:
from os import listdir
from os.path import isfile, join

fpath = os.path.join(ROOT_DIR, "risei-history/risei-3d-cnn-parameters")

files = [f for f in listdir(fpath) if isfile(join(fpath, f))]
files[:5]

['hmap-parameters--deletion--b1+0-b2+0-b2+0.cls',
 'hmap-parameters--deletion--b1+0-b2+0-b2+1.cls',
 'hmap-parameters--deletion--b1+0-b2+0.25-b2+0.cls',
 'hmap-parameters--deletion--b1+0-b2+0.25-b2+1.cls',
 'hmap-parameters--deletion--b1+0-b2+0.5-b2+0.cls']

In [6]:
import re


def parse(fname):
    p = re.compile("^hmap-parameters--(\w+)--b1\+(\d+[.]?\d*)-b2\+(\d+[.]?\d*)-b2\+(\d+[.]?\d*)\.cls$")
    return p.match(fname).groups()

print(parse('hmap-parameters--deletion--b1+0-b2+0.5-b2+0.cls'))

('deletion', '0', '0.5', '0')


In [7]:
from src.heatmaps.evaluation import HeatmapEvaluationHistory

data = {}


def append(key, value):
    if not key in data:
        data[key] = []
    data[key].append(value)

    
for fname in files:
    metric, b1, b2, b2_value = parse(fname)
    append('metric', metric)
    append('b1', float(b1))
    append('b2', float(b2))
    append('b2_value', float(b2_value))
    
#     print(f"loading {fname}...")
    history = HeatmapEvaluationHistory.load(fpath, fname[:-4])
    desc = history._description()
    for key, value in desc.items():
        append(key, value)
    
    
df = pd.DataFrame(data=data)
df.head()

,metric,b1,b2,b2_value,heatmaps,auc_mean,auc_p25,auc_median,auc_p75,auc_max,auc_min,auc_std
0,deletion,0.0,0.00,0.0,20,0.594591,0.562278,0.595900,0.642628,0.763560,0.406783,0.077598
1,deletion,0.0,0.00,1.0,20,0.594591,0.562278,0.595900,0.642628,0.763560,0.406783,0.077598
2,deletion,0.0,0.25,0.0,20,0.596732,0.564075,0.597532,0.643076,0.767384,0.405720,0.077990
3,deletion,0.0,0.25,1.0,20,0.540205,0.474668,0.545324,0.590410,0.750998,0.370201,0.092026
4,deletion,0.0,0.50,0.0,20,0.603527,0.577062,0.608133,0.646452,0.771426,0.407250,0.079088


In [8]:
def table(metric, value):
    df_m = df[df['metric'] == metric]
    return pd.pivot_table(df_m, values=value, index=["b1"], columns=["b2", "b2_value"], fill_value=0)

In [9]:
table('deletion', 'auc_median')

b2            0.00                0.25                0.50            \
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.595900  0.595900  0.597532  0.545324  0.608133  0.521814   
0.25      0.617166  0.617166  0.597817  0.545592  0.608281  0.521619   
0.50      0.627478  0.000000  0.000000  0.000000  0.608954  0.521469   
1.00      0.638605  0.000000  0.000000  0.000000  0.609829  0.521318   

b2            0.75                1.00            
b2_value       0.0       1.0       0.0       1.0  
b1                                                
0.00      0.633929  0.537904  0.666324  0.527962  
0.25      0.633507  0.537747  0.000000  0.000000  
0.50      0.000000  0.000000  0.000000  0.000000  
1.00      0.000000  0.000000  0.000000  0.000000

In [10]:
table('insertion', 'auc_median')

b2            0.00                0.25                0.50            \
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.565903  0.565903  0.559735  0.633579  0.542227  0.647707   
0.25      0.535090  0.535090  0.559925  0.633470  0.542677  0.647959   
0.50      0.512993  0.000000  0.000000  0.000000  0.542262  0.648216   
1.00      0.499336  0.000000  0.000000  0.000000  0.542333  0.648316   

b2            0.75                1.00            
b2_value       0.0       1.0       0.0       1.0  
b1                                                
0.00      0.506352  0.620739  0.447044  0.622947  
0.25      0.506335  0.620737  0.000000  0.000000  
0.50      0.000000  0.000000  0.000000  0.000000  
1.00      0.000000  0.000000  0.000000  0.000000

In [11]:
df_m = df.copy().sort_values("b1")
pd.pivot_table(df_m, values="auc_median", index=["b1"], columns=["metric", "b2", "b2_value"], fill_value=0)

metric    deletion                                                    \
b2            0.00                0.25                0.50             
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.595900  0.595900  0.597532  0.545324  0.608133  0.521814   
0.25      0.617166  0.617166  0.597817  0.545592  0.608281  0.521619   
0.50      0.627478  0.000000  0.000000  0.000000  0.608954  0.521469   
1.00      0.638605  0.000000  0.000000  0.000000  0.609829  0.521318   

metric                                           insertion            \
b2            0.75                1.00                0.00             
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.633929  0.537904  0.666324  0.527962  0.565903  0.565903   
0.25      0.633507  0.537747  0.000000  0.000000  0.535090  0.535090   
0.50      0.000000  0.000000  0.000000  0.000000  0.512993  0.000000   
1.00      0.000000  0.000000  0.000000  0.000000  0.499336  0.000000   

metric                                                                \
b2            0.25                0.50                0.75             
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.559735  0.633579  0.542227  0.647707  0.506352  0.620739   
0.25      0.559925  0.633470  0.542677  0.647959  0.506335  0.620737   
0.50      0.000000  0.000000  0.542262  0.648216  0.000000  0.000000   
1.00      0.000000  0.000000  0.542333  0.648316  0.000000  0.000000   

metric                        
b2            1.00            
b2_value       0.0       1.0  
b1                            
0.00      0.447044  0.622947  
0.25      0.000000  0.000000  
0.50      0.000000  0.000000  
1.00      0.000000  0.000000

In [12]:
df_m = df.copy().sort_values("b1")
pd.pivot_table(df_m, values="auc_median", index=["b1"], columns=["metric", "b2", "b2_value"], fill_value=0)

metric    deletion                                                    \
b2            0.00                0.25                0.50             
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.595900  0.595900  0.597532  0.545324  0.608133  0.521814   
0.25      0.617166  0.617166  0.597817  0.545592  0.608281  0.521619   
0.50      0.627478  0.000000  0.000000  0.000000  0.608954  0.521469   
1.00      0.638605  0.000000  0.000000  0.000000  0.609829  0.521318   

metric                                           insertion            \
b2            0.75                1.00                0.00             
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.633929  0.537904  0.666324  0.527962  0.565903  0.565903   
0.25      0.633507  0.537747  0.000000  0.000000  0.535090  0.535090   
0.50      0.000000  0.000000  0.000000  0.000000  0.512993  0.000000   
1.00      0.000000  0.000000  0.000000  0.000000  0.499336  0.000000   

metric                                                                \
b2            0.25                0.50                0.75             
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.559735  0.633579  0.542227  0.647707  0.506352  0.620739   
0.25      0.559925  0.633470  0.542677  0.647959  0.506335  0.620737   
0.50      0.000000  0.000000  0.542262  0.648216  0.000000  0.000000   
1.00      0.000000  0.000000  0.542333  0.648316  0.000000  0.000000   

metric                        
b2            1.00            
b2_value       0.0       1.0  
b1                            
0.00      0.447044  0.622947  
0.25      0.000000  0.000000  
0.50      0.000000  0.000000  
1.00      0.000000  0.000000

In [13]:
col = 'auc_median'
df_m = df.copy().sort_values('b1')
def apply_fn(row):
    if row['metric'] == 'deletion':
        return 1 - row[col]
    return row[col]
df_m[col] = df.apply(apply_fn, axis=1)

pd.pivot_table(df_m, values=col, index=['b1'], columns=['b2', 'b2_value'], fill_value=np.nan)

b2            0.00                0.25                0.50            \
b2_value       0.0       1.0       0.0       1.0       0.0       1.0   
b1                                                                     
0.00      0.485002  0.485002  0.481101  0.544128  0.467047  0.562946   
0.25      0.458962  0.458962  0.481054  0.543939  0.467198  0.563170   
0.50      0.442757       NaN       NaN       NaN  0.466654  0.563374   
1.00      0.430366       NaN       NaN       NaN  0.466252  0.563499   

b2            0.75               1.00            
b2_value       0.0       1.0      0.0       1.0  
b1                                               
0.00      0.436211  0.541418  0.39036  0.547493  
0.25      0.436414  0.541495      NaN       NaN  
0.50           NaN       NaN      NaN       NaN  
1.00           NaN       NaN      NaN       NaN